## Google Colab Ortamını Hazırlama

In [ ]:
!pip install torch transformers datasets tqdm sentencepiece sacremoses

## Veri Setini Yükleme ve Ön İşleme

In [ ]:
import pandas as pd
import json

# JSONL dosyasının bulunduğu yolu belirleyin
data_path = "/content/data.jsonl"  # Dosyanın yolu
data = []

with open(data_path, "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

# Pandas DataFrame'e dönüştürme
df = pd.DataFrame(data)

# İlk birkaç satırı yazdırma
print(df.head())


                                                  tr  \
0            Devenin belini kıran son saman çöpüdür.   
1                                        Bilmiyorum.   
2                           Mum kendiliğinden söndü.   
3  On, on bir, on iki, on üç, on dört, on beş, on...   
4  Ek olarak yaşlılar birbirleriyle sosyalleşebil...   

                                                  en  
0            The last straw breaks the camel's back.  
1                                      I don't know.  
2                     The candle went out by itself.  
3  Ten, eleven, twelve, thirteen, fourteen, fifte...  
4  In addition many groups have been formed so th...  


## Tokenizer Seçimi ve Veri Setinin Tokenize Edilmesi

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Tokenizer ve modeli doğrudan yükleyelim
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tr-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tr-en")

# Örnek tokenizasyon
example_tr = df["tr"].iloc[0]  # İlk Türkçe cümle
example_en = df["en"].iloc[0]  # İlk İngilizce cümle

tokens_tr2en = tokenizer(example_tr, return_tensors="pt")
tokens_en2tr = tokenizer(example_en, return_tensors="pt")

# Tokenları yazdıralım
print("Türkçeden İngilizceye Tokenler:", tokens_tr2en["input_ids"])
print("İngilizceden Türkçeye Tokenler:", tokens_en2tr["input_ids"])


Türkçeden İngilizceye Tokenler: tensor([[31415,   187,  8922,   692,  3286,   373,   283, 29271, 33294,  2453,
             2,     0]])
İngilizceden Türkçeye Tokenler: tensor([[  71, 2169, 1517,  125, 9376,  985,  125, 3763,   47, 3739,    9, 6374,
          977,    4,    8,  125, 5544,    2,    0]])


## Veri Setini Model İçin Hazırlama

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_lang="tr", target_lang="en", max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.source_lang = source_lang
        self.target_lang = target_lang
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_text = self.data[self.source_lang].iloc[idx]  # Kaynak dildeki cümle
        tgt_text = self.data[self.target_lang].iloc[idx]  # Hedef dildeki çeviri

        # Tokenize et
        src_encoding = self.tokenizer(src_text, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")
        tgt_encoding = self.tokenizer(tgt_text, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")

        # PyTorch için tensörleri döndür
        return {
            "input_ids": src_encoding["input_ids"].squeeze(0),
            "attention_mask": src_encoding["attention_mask"].squeeze(0),
            "labels": tgt_encoding["input_ids"].squeeze(0)
        }

# Veri setini oluştur
train_dataset = TranslationDataset(df, tokenizer)

# DataLoader: Modeli eğitirken kullanacağız
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Bir örnek alıp inceleyelim
batch = next(iter(train_dataloader))
print("Input IDs:", batch["input_ids"].shape)
print("Labels:", batch["labels"].shape)


Input IDs: torch.Size([16, 128])
Labels: torch.Size([16, 128])


## Modeli Eğitme

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62389, 512, padding_idx=62388)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62389, 512, padding_idx=62388)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [ ]:
from tqdm import tqdm
from torch.optim import AdamW
from transformers import get_scheduler

# Modeli eğitime alalım
model.train()

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Öğrenme oranı scheduler'ı
num_training_steps = len(train_dataloader) * 3  # 3 epoch
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# GPU'yu kullanalım
model.to(device)

# Eğitim döngüsü
num_epochs = 3  # 3 epoch çalıştıracağız
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # tqdm ile batch'ler üzerinde ilerleme çubuğu ekleyelim
    progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}", total=len(train_dataloader))

    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Modelin tahminini al
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss  # Hata hesapla
        loss.backward()  # Geri yayılım (backpropagation)

        optimizer.step()  # Ağırlıkları güncelle
        lr_scheduler.step()  # Öğrenme oranını güncelle
        optimizer.zero_grad()  # Gradient'leri sıfırla

        # İlerleme çubuğunu güncelle
        progress_bar.set_postfix(loss=loss.item())

    print(f"Loss for epoch {epoch+1}: {loss.item():.4f}")


Epoch 1/3


Görüntülenen çıkış son 5000 satıra kısaltıldı.
Training Epoch 1:  25%|██▌       | 11195/44303 [1:06:36<3:16:59,  2.80it/s, loss=0.0673]


KeyboardInterrupt: 

## Eğitilmiş Modeli Kaydetme

### Modeli ve Tokenizer’ı Kaydetme

In [ ]:
import os

save_path = "/content/translation_model_tr2en"  # Kaydetmek istediğin dizini belirle
os.makedirs(save_path, exist_ok=True)

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model ve tokenizer şu dizine kaydedildi: {save_path}")


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62388]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Model ve tokenizer şu dizine kaydedildi: /content/translation_model_tr2en


## Modeli Test Etme

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Kaydedilen modeli yükleyelim
model_path = "/content/translation_model_tr2en"  # Eğer Google Drive'a kaydettiysen path'i değiştir
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Modeli GPU'ya taşıyalım
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62389, 512, padding_idx=62388)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62389, 512, padding_idx=62388)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

### Model İndirme

In [ ]:
import shutil
from google.colab import files

# Kaydedilen modelin bulunduğu dizini sıkıştırıyoruz
shutil.make_archive('/content/translation_model_tr2en.zip', 'zip', save_path)

# Dosyayı bilgisayarınıza indirmek için
files.download('/content/translation_model_tr2en.zip')

FileNotFoundError: Cannot find file: /content/translation_model_tr2en.zip

## Test Cümleleri

In [22]:
def translate(text, model, tokenizer):
    model.eval()  # Modeli evaluation moduna al
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test cümleleri
test_sentences = [
    "Merhaba, nasılsın?",
    "Bu model Türkçeden İngilizceye çeviri yapabiliyor.",
    "Bugün hava çok güzel.",
    "Yapay zeka geleceğin teknolojisidir.",
    "Seni seviyorum."
]

for sentence in test_sentences:
    translation = translate(sentence, model, tokenizer)
    print(f"Türkçe: {sentence} \nİngilizce: {translation}\n")

Türkçe: Merhaba, nasılsın? 
İngilizce: Hi, how are you?

Türkçe: Bu model Türkçeden İngilizceye çeviri yapabiliyor. 
İngilizce: This model can translate from Turkish to English.

Türkçe: Bugün hava çok güzel. 
İngilizce: It's a beautiful day today.

Türkçe: Yapay zeka geleceğin teknolojisidir. 
İngilizce: Artificial intelligence is the technology of the future.

Türkçe: Seni seviyorum.merhaba 
İngilizce: I love you. Hi.

